In [1]:
import random
import numpy as np
import torch
import os
import pathlib
import pickle
from gurobipy import *
from rsome import ro
from rsome import grb_solver as grb
import rsome as rso
from rsome import cpt_solver as cpt
import pandas as pd
torch.manual_seed(42)
torch.cuda.manual_seed(42)

from Performance import performance_evaluation
perfs = performance_evaluation()

from Network import network_design
Network = network_design()

In [2]:
def Prepare_Data(DataPath,lower, upper, p, d, coef_seed,iteration_all,num_test, num_train, alpha,mis,data_generation_process,x_dist, e_dist, x_low, x_up, x_mean, x_var, bump):
# #  ****** Coef generation *********
    from Data import data_generation
    data_gen = data_generation()
    # W_star = data_gen.generate_truth(DataPath,lower, upper, p, d, coef_seed,data_generation_process) 
    # print("W_star = ",W_star[0,:])
    np.random.seed(coef_seed)
    x_test_all = {}; c_test_all = {}; x_train_all = {}; c_train_all = {}; W_star_all = {}; noise_train_all = {}; noise_test_all = {}
    for iter in iteration_all:
        DataPath_iter = DataPath +"iter="+str(iter)+"/"
        pathlib.Path(DataPath_iter).mkdir(parents=True, exist_ok=True)
        W_star = data_gen.generate_truth(DataPath_iter,lower, upper, p, d, iter,data_generation_process) 
        # #  ****** Data generation *********
        x_test_all[iter], c_test_all[iter], x_train_all[iter], c_train_all[iter], noise_train_all[iter],noise_test_all[iter],W_star_all[iter] = data_gen.generate_samples(iter,DataPath_iter,p, d, num_test, num_train, alpha, W_star, mis, num_test, 
                                data_generation_process, x_dist, e_dist, x_low, x_up, x_mean, x_var, bump) 
        # print()
    return x_test_all, c_test_all, x_train_all, c_train_all, noise_train_all,noise_test_all,W_star_all

In [3]:
def Implement_Oracle(arcs, grid,mis,bump,W_star_all,x_test_all,noise_test_all,iteration_all,num_feat,data_generation_process):
    cost_Oracle_with_noise_all = {}; cost_Oracle_wo_noise_all = {}
    for iter in iteration_all:
        if data_generation_process == "SPO_Data_Generation":
            cost_oracle_ori = (W_star_all[iter] @ x_test_all[iter].T)/np.sqrt(num_feat) + 3
            non_negative_cols = (cost_oracle_ori > 0).all(axis=0)
            cost_oracle_ori = cost_oracle_ori[:,non_negative_cols]

            cost_oracle_pred = (cost_oracle_ori ** mis + 1).T
            cost_Oracle_with_noise_all[iter] = perfs.compute_SPO_out_of_sample_Cost(arcs, grid,cost_oracle_pred,cost_oracle_pred,noise_test_all[iter])
            if iter % 20 == 0 and iter > 0:
                print("Oracle: iter=",iter,",cost_Oracle_with_noise_all=",np.nanmean(cost_Oracle_with_noise_all[iter]))

        if data_generation_process == "DDR_Data_Generation":
            cost_oracle_ori = (W_star_all[iter] @ x_test_all[iter].T) + bump
            cost_oracle_pred = (cost_oracle_ori ** mis).T
            cost_Oracle_with_noise_all[iter] = perfs.compute_DDR_out_of_sample_Cost(arcs, grid,cost_oracle_pred,cost_oracle_pred,noise_test_all[iter],True)
            cost_Oracle_wo_noise_all[iter] = perfs.compute_DDR_out_of_sample_Cost(arcs, grid,cost_oracle_pred,cost_oracle_pred,noise_test_all[iter],False)

            # print("Oracle: iter=",iter,",cost_Oracle_with_noise_all=",np.nanmean(cost_Oracle_with_noise_all[iter]),",cost_Oracle_wo_noise_all=",np.nanmean(cost_Oracle_wo_noise_all[iter]))
    return cost_Oracle_with_noise_all,cost_Oracle_wo_noise_all

In [4]:
def Implement_OLS(arcs, grid,mis,bump,W_star_all,x_test_all,noise_test_all,x_train_all,c_train_all,iteration_all,num_feat,data_generation_process):
    from OLS import ols_method
    ols_method_obj = ols_method()
    W_ols_all = {}; w0_ols_all = {}; t_ols_all = {}; obj_ols_all = {}
    cost_OLS_with_noise_all = {}; cost_OLS_wo_noise_all = {}
    for iter in iteration_all:
        # compute OLS performance
        W_ols_all[iter], w0_ols_all[iter], t_ols_all[iter], obj_ols_all[iter] = ols_method_obj.ols_solver("",x_train_all[iter], c_train_all[iter])
        cost_dem = (W_ols_all[iter] @ x_test_all[iter].T).T + w0_ols_all[iter]

        if data_generation_process == "SPO_Data_Generation":
            cost_oracle_ori = (W_star_all[iter] @ x_test_all[iter].T)/np.sqrt(num_feat) + 3
            non_negative_cols = (cost_oracle_ori > 0).all(axis=0)
            cost_oracle_ori = cost_oracle_ori[:,non_negative_cols]
            cost_dem = cost_dem[non_negative_cols,:]
            
            cost_oracle_pred = (cost_oracle_ori ** mis + 1).T
            cost_OLS_with_noise_all[iter] = perfs.compute_SPO_out_of_sample_Cost(arcs, grid,cost_dem,cost_oracle_pred,noise_test_all[iter])

        if data_generation_process == "DDR_Data_Generation":
            cost_oracle_ori = (W_star_all[iter] @ x_test_all[iter].T) + bump
            cost_oracle_pred = (cost_oracle_ori ** mis).T
            cost_OLS_with_noise_all[iter] = perfs.compute_DDR_out_of_sample_Cost(arcs, grid,cost_dem,cost_oracle_pred,noise_test_all[iter],True)
            cost_OLS_wo_noise_all[iter] = perfs.compute_DDR_out_of_sample_Cost(arcs, grid,cost_dem,cost_oracle_pred,noise_test_all[iter],False)
        if iter % 20 == 0 and iter > 0:
            print("OLS: iter=",iter,",cost_OLS_with_noise_all =",np.nanmean(cost_OLS_with_noise_all[iter]))
    return cost_OLS_with_noise_all,cost_OLS_wo_noise_all

In [5]:
def Implement_DDR(mu_all,lamb_all,arcs, grid,mis,bump,W_star_all,x_test_all,noise_test_all,x_train_all,c_train_all,iteration_all,num_feat,data_generation_process):
    from DDR import DDR_method
    ddr_object = DDR_method()
    num_nodes = grid[0] * grid[0]

    w0_ddr_dict = {}; W_ddr_dict = {}
    cost_DDR_with_noise_all = {}; cost_DDR_wo_noise_all = {}
    for iter in iteration_all:
        for mu in mu_all:
            for lamb in lamb_all:
                w0_ddr_dict[iter,mu,lamb],W_ddr_dict[iter,mu,lamb],alpha_rst,obj_ddr = ddr_object.solve_DDR(arcs,lamb,mu,num_nodes,x_train_all[iter],c_train_all[iter])
                cost_pred = (W_ddr_dict[iter,mu,lamb] @ x_test_all[iter].T).T + w0_ddr_dict[iter,mu,lamb]

                if data_generation_process == "SPO_Data_Generation":
                    cost_oracle_ori = (W_star_all[iter] @ x_test_all[iter].T)/np.sqrt(num_feat) + 3
                    non_negative_cols = (cost_oracle_ori > 0).all(axis=0)
                    cost_oracle_ori = cost_oracle_ori[:,non_negative_cols]
                    cost_oracle_pred = (cost_oracle_ori ** mis + 1).T
                    
                    cost_pred = cost_pred[non_negative_cols,:]
            
                    cost_DDR_with_noise_all[iter,mu,lamb] = perfs.compute_SPO_out_of_sample_Cost(arcs, grid,cost_pred,cost_oracle_pred,noise_test_all[iter])

                if data_generation_process == "DDR_Data_Generation":
                    cost_oracle_ori = (W_star_all[iter] @ x_test_all[iter].T) + bump
                    cost_oracle_pred = (cost_oracle_ori ** mis).T
                    # cost_DDR_with_noise_all[iter,mu,lamb] = perfs.compute_DDR_out_of_sample_Cost(arcs, grid,cost_pred,cost_oracle_pred,noise_test_all[iter],True)
                    cost_DDR_wo_noise_all[iter,mu,lamb] = perfs.compute_DDR_out_of_sample_Cost(arcs, grid,cost_pred,cost_oracle_pred,noise_test_all[iter],False)
        if iter % 20 == 0 and iter > 0:
            print("DDR: iter=",iter,",mu=",mu,",lamb=",lamb,",cost_DDR_with_noise_all =",np.nanmean(cost_DDR_with_noise_all[iter,mu,lamb]))

    return cost_DDR_with_noise_all,cost_DDR_wo_noise_all

In [ ]:
def Implement_EPO(DataPath,iteration_all,batch_size,num_epochs,method_names,W_star_all,bump,x_train_all,c_train_all,x_test_all,noise_test_all,\
                  arcs,grid,epo_runner,perfs,num_feat,mis,data_generation_process):
    W_EPO_all = {}; w0_EPO_all = {}
    cost_EPO_all = {}
    for iter in iteration_all:
        DataPath_seed = DataPath +"iter="+str(iter)+"/"
        pathlib.Path(DataPath_seed).mkdir(parents=True, exist_ok=True)
        W_EPO_all[iter],w0_EPO_all[iter] = epo_runner.run(method_names,DataPath_seed,batch_size,num_feat,grid,num_epochs,\
                                        x_train_all[iter],c_train_all[iter],arcs)
        
        cost_pred = (W_EPO_all[iter] @ x_test_all[iter].T).T + w0_EPO_all[iter]
        if data_generation_process == "SPO_Data_Generation":
            cost_oracle_ori = (W_star_all[iter] @ x_test_all[iter].T)/np.sqrt(num_feat) + 3
            non_negative_cols = (cost_oracle_ori > 0).all(axis=0)
            cost_oracle_ori = cost_oracle_ori[:,non_negative_cols]
            cost_oracle_pred = (cost_oracle_ori ** mis + 1).T
            
            cost_pred = cost_pred[non_negative_cols,:]
            cost_EPO_all[iter] = perfs.compute_SPO_out_of_sample_Cost(arcs, grid,cost_pred,cost_oracle_pred,noise_test_all[iter])

        if data_generation_process == "DDR_Data_Generation":
            cost_oracle_ori = (W_star_all[iter] @ x_test_all[iter].T) + bump
            cost_oracle_pred = (cost_oracle_ori ** mis).T
            cost_EPO_all[iter] = perfs.compute_DDR_out_of_sample_Cost(arcs, grid,cost_pred,cost_oracle_pred,noise_test_all[iter])
        print(method_names[0],": iter=",iter,",cost=",np.nanmean(cost_EPO_all[iter]))

    return W_EPO_all,w0_EPO_all,cost_EPO_all

In [6]:
def cross_compare2plus(c_item, c_base, c_oracle):
    N = len(c_item)
    c_diff = c_base - c_item
    lbel = np.zeros((N,1))
    
    equals = np.sum(c_diff == 0)
    wins = np.sum(c_diff > 0) # indicate num of c_item is lower than c_base
    lose = np.sum(c_diff < 0)
    
    lbel[c_diff < 0] = 1
    lbel[c_diff > 0] = -1
    
#     print(N, equals, wins, lose)
    if N == equals:
        win_ratio = 0.5
    else:
        win_ratio = wins/(N - equals)
    cost_reduction = (np.mean(c_diff))/np.abs(np.mean(c_base))
    regret_reduction = (np.mean(c_diff))/np.abs(np.mean(c_base) - np.mean(c_oracle))
    return win_ratio, cost_reduction, regret_reduction

# Parameters

In [7]:
grid = (2,2) # grid size
arcs,arc_index_mapping = Network._getArcs(grid)

num_test = 1000
lower = 0 # coef lower bound
upper = 1 # coef upper bound
d = (grid[0] - 1) * (grid[0] - 1) * 2 + 2 * (grid[0] - 1) # num of arcs
num_nodes = grid[0]*grid[0]
coef_seed = 1

x_dist = 'uniform'
e_dist = 'normal'
x_low = -2
x_up = 2
x_mean = 2
x_var = 2
bump = 100
data_generation_process = "SPO_Data_Generation"
# data_generation_process = "DDR_Data_Generation"

In [8]:
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
grandparent_directory = os.path.dirname(parent_directory)
DataPath_parent = os.path.dirname(grandparent_directory) + '/Data/Shortest_Path_Reproduce/Various_Settings/'+str(grid[0])+'by'+str(grid[1])+'_grid_' + data_generation_process + "/"
pathlib.Path(DataPath_parent).mkdir(parents=True, exist_ok=True)

### Impact of sample size

In [9]:
num_feat = 5 # size of feature
p = num_feat

deg = 1.0 # polynomial degree
mis = deg # model misspecification

e = 0.5 # scale of normal std or the range of uniform. For the error term
alpha = e # scale of normal std or the range of uniform. For the error term

iteration_all = np.arange(0,100)

num_train_all = [50,100,200,500,1000]
mu_all = np.asarray([0.5])
lamb_all = np.asarray([0.3])
h2h_ddr_vs_ols_all = {}; cost_reduction_ddr_vs_ols_all = {}; regret_reduction_ddr_vs_ols_all = {}
for num_train in num_train_all:
    DataPath = DataPath_parent + "data_size="+str(num_train)+"_deg="+str(deg)+"_e="+str(e)+"_d="+str(d)+"_p="+str(p)+"_x_dist="+x_dist+"_num_test="+str(num_test)+"_diff_W/"
    pathlib.Path(DataPath).mkdir(parents=True, exist_ok=True)
    print("DataPath:", DataPath)
    x_test_all, c_test_all, x_train_all, c_train_all,noise_train_all,noise_test_all,W_star_all = Prepare_Data(DataPath,lower, upper, p, d, coef_seed,iteration_all,num_test, num_train, alpha,mis,data_generation_process,x_dist, e_dist, x_low, x_up, x_mean, x_var, bump)
    cost_Oracle_with_noise_all,cost_Oracle_wo_noise_all = Implement_Oracle(arcs, grid,mis,bump,W_star_all,x_test_all,noise_test_all,iteration_all,num_feat,data_generation_process)
    cost_OLS_with_noise_all,cost_OLS_wo_noise_all = Implement_OLS(arcs, grid,mis,bump,W_star_all,x_test_all,noise_test_all,x_train_all,c_train_all,iteration_all,num_feat,data_generation_process)
    cost_DDR_with_noise_all,cost_DDR_wo_noise_all = Implement_DDR(mu_all,lamb_all,arcs, grid,mis,bump,W_star_all,x_test_all,noise_test_all,x_train_all,c_train_all,iteration_all,num_feat,data_generation_process)
    with open(DataPath+'cost_OLS_with_noise_all.pkl', "wb") as tf:
        pickle.dump(cost_OLS_with_noise_all,tf)
    with open(DataPath+'cost_Oracle_with_noise_all.pkl', "wb") as tf:
        pickle.dump(cost_Oracle_with_noise_all,tf)
    with open(DataPath+'cost_DDR_with_noise_all.pkl', "wb") as tf:
        pickle.dump(cost_DDR_with_noise_all,tf)

DataPath: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data/Shortest_Path_Reproduce/Various_Settings/2by2_grid_SPO_Data_Generation/data_size=50_deg=1.0_e=0.5_d=4_p=5_x_dist=uniform_num_test=1000_diff_W/
Set parameter Username
Academic license - for non-commercial use only - expires 2026-03-13
Oracle: iter= 20 ,cost_Oracle_with_noise_all= 7.655407981628993
Oracle: iter= 40 ,cost_Oracle_with_noise_all= 7.7712673991470815
Oracle: iter= 60 ,cost_Oracle_with_noise_all= 7.765293840348416
Oracle: iter= 80 ,cost_Oracle_with_noise_all= 7.601360162654932
OLS: iter= 20 ,cost_OLS_with_noise_all = 7.708650868106933
OLS: iter= 40 ,cost_OLS_with_noise_all = 7.909320728378931
OLS: iter= 60 ,cost_OLS_with_noise_all = 7.793295944051177
OLS: iter= 80 ,cost_OLS_with_noise_all = 7.676628260689021
DDR: iter= 20 ,mu= 0.5 ,lamb= 0.3 ,cost_DDR_with_noise_all = 7.713802698868517
DDR: iter= 40 ,mu= 0.5 ,lamb= 0.3 ,cost_DDR_with_noise_all = 7.909823262648006
DDR: iter= 60 ,mu= 0.5 ,lamb= 0.3 ,c

In [10]:
mu = mu_all[0]
lamb = lamb_all[0]

regret_N = np.zeros(len(num_train_all))
h2h_N = np.zeros(len(num_train_all))
_index = 0
for num_train in num_train_all:
    DataPath = DataPath_parent + "data_size="+str(num_train)+"_deg="+str(deg)+"_e="+str(e)+"_d="+str(d)+"_p="+str(p)+"_x_dist="+x_dist+"_num_test="+str(num_test)+"_diff_W/"
    print(DataPath)
    with open(DataPath+'cost_OLS_with_noise_all.pkl', "rb") as tf:
        cost_OLS_with_noise_all = pickle.load(tf)
    with open(DataPath+'cost_Oracle_with_noise_all.pkl', "rb") as tf:
        cost_Oracle_with_noise_all = pickle.load(tf)
    with open(DataPath+'cost_DDR_with_noise_all.pkl', "rb") as tf:
        cost_DDR_with_noise_all = pickle.load(tf)

    h2h_ddr_ols = np.zeros(len(iteration_all)); cost_reduction_ddr_vs_ols = np.zeros(len(iteration_all)); regret_reduction_ddr_vs_ols = np.zeros(len(iteration_all))
    for iter_index in range(len(iteration_all)):
        iter = iteration_all[iter_index]
        h2h_ddr_ols[iter_index],cost_reduction_ddr_vs_ols[iter_index],regret_reduction_ddr_vs_ols[iter_index] = cross_compare2plus(cost_DDR_with_noise_all[iter,mu,lamb], cost_OLS_with_noise_all[iter], cost_Oracle_with_noise_all[iter])
    regret_N[_index] = np.round( len(regret_reduction_ddr_vs_ols[regret_reduction_ddr_vs_ols > 0.0])/len(regret_reduction_ddr_vs_ols),4 )
    h2h_N[_index] = np.round( len(h2h_ddr_ols[h2h_ddr_ols >= 0.5])/len(h2h_ddr_ols),4 )
    _index = _index + 1

/Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data/Shortest_Path_Reproduce/Various_Settings/2by2_grid_SPO_Data_Generation/data_size=50_deg=1.0_e=0.5_d=4_p=5_x_dist=uniform_num_test=1000_diff_W/
/Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data/Shortest_Path_Reproduce/Various_Settings/2by2_grid_SPO_Data_Generation/data_size=100_deg=1.0_e=0.5_d=4_p=5_x_dist=uniform_num_test=1000_diff_W/
/Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data/Shortest_Path_Reproduce/Various_Settings/2by2_grid_SPO_Data_Generation/data_size=200_deg=1.0_e=0.5_d=4_p=5_x_dist=uniform_num_test=1000_diff_W/
/Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data/Shortest_Path_Reproduce/Various_Settings/2by2_grid_SPO_Data_Generation/data_size=500_deg=1.0_e=0.5_d=4_p=5_x_dist=uniform_num_test=1000_diff_W/
/Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data/Shortest_Path_Reproduce/Various_Settings/2by2_grid_SPO_Data_Generation/data_size

### Impact of num of features

In [11]:
deg = 1.0 # polynomial degree
mis = deg # model misspecification

e = 0.5 # scale of normal std or the range of uniform. For the error term
alpha = e # scale of normal std or the range of uniform. For the error term

num_train = 100

num_feat_all = [1,3,5,7,10,15]
h2h_ddr_vs_ols_all = {}; cost_reduction_ddr_vs_ols_all = {}; regret_reduction_ddr_vs_ols_all = {}
for num_feat in num_feat_all:
    p = num_feat
    if p != 5:
        DataPath = DataPath_parent + "data_size="+str(num_train)+"_deg="+str(deg)+"_e="+str(e)+"_d="+str(d)+"_p="+str(p)+"_x_dist="+x_dist+"_num_test="+str(num_test)+"_diff_W/"
        pathlib.Path(DataPath).mkdir(parents=True, exist_ok=True)
        print("DataPath:", DataPath)
        x_test_all, c_test_all, x_train_all, c_train_all,noise_train_all,noise_test_all,W_star_all = Prepare_Data(DataPath,lower, upper, p, d, coef_seed,iteration_all,num_test, num_train, alpha,mis,data_generation_process,x_dist, e_dist, x_low, x_up, x_mean, x_var, bump)
        cost_Oracle_with_noise_all,cost_Oracle_wo_noise_all = Implement_Oracle(arcs, grid,mis,bump,W_star_all,x_test_all,noise_test_all,iteration_all,num_feat,data_generation_process)
        cost_OLS_with_noise_all,cost_OLS_wo_noise_all = Implement_OLS(arcs, grid,mis,bump,W_star_all,x_test_all,noise_test_all,x_train_all,c_train_all,iteration_all,num_feat,data_generation_process)
        cost_DDR_with_noise_all,cost_DDR_wo_noise_all = Implement_DDR(mu_all,lamb_all,arcs, grid,mis,bump,W_star_all,x_test_all,noise_test_all,x_train_all,c_train_all,iteration_all,num_feat,data_generation_process)
        with open(DataPath+'cost_OLS_with_noise_all.pkl', "wb") as tf:
            pickle.dump(cost_OLS_with_noise_all,tf)
        with open(DataPath+'cost_Oracle_with_noise_all.pkl', "wb") as tf:
            pickle.dump(cost_Oracle_with_noise_all,tf)
        with open(DataPath+'cost_DDR_with_noise_all.pkl', "wb") as tf:
            pickle.dump(cost_DDR_with_noise_all,tf)

DataPath: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data/Shortest_Path_Reproduce/Various_Settings/2by2_grid_SPO_Data_Generation/data_size=100_deg=1.0_e=0.5_d=4_p=1_x_dist=uniform_num_test=1000_diff_W/
Oracle: iter= 20 ,cost_Oracle_with_noise_all= 8.017179915527835
Oracle: iter= 40 ,cost_Oracle_with_noise_all= 7.64565857791319
Oracle: iter= 60 ,cost_Oracle_with_noise_all= 7.6203659771262755
Oracle: iter= 80 ,cost_Oracle_with_noise_all= 7.617115912846993
OLS: iter= 20 ,cost_OLS_with_noise_all = 8.025949489084745
OLS: iter= 40 ,cost_OLS_with_noise_all = 7.656250456136739
OLS: iter= 60 ,cost_OLS_with_noise_all = 7.620782501119949
OLS: iter= 80 ,cost_OLS_with_noise_all = 7.640686326733859
DDR: iter= 20 ,mu= 0.5 ,lamb= 0.3 ,cost_DDR_with_noise_all = 8.025589900823295
DDR: iter= 40 ,mu= 0.5 ,lamb= 0.3 ,cost_DDR_with_noise_all = 7.655787065094626
DDR: iter= 60 ,mu= 0.5 ,lamb= 0.3 ,cost_DDR_with_noise_all = 7.620479015791167
DDR: iter= 80 ,mu= 0.5 ,lamb= 0.3 ,cost_DDR_with

In [12]:
mu = mu_all[0]
lamb = lamb_all[0]

regret_p = np.zeros(len(num_feat_all))
h2h_p = np.zeros(len(num_feat_all))
_index = 0
for num_feat in num_feat_all:
    p = num_feat
    DataPath = DataPath_parent + "data_size="+str(num_train)+"_deg="+str(deg)+"_e="+str(e)+"_d="+str(d)+"_p="+str(p)+"_x_dist="+x_dist+"_num_test="+str(num_test)+"_diff_W/"
    print(DataPath)
    with open(DataPath+'cost_OLS_with_noise_all.pkl', "rb") as tf:
        cost_OLS_with_noise_all = pickle.load(tf)
    with open(DataPath+'cost_Oracle_with_noise_all.pkl', "rb") as tf:
        cost_Oracle_with_noise_all = pickle.load(tf)
    with open(DataPath+'cost_DDR_with_noise_all.pkl', "rb") as tf:
        cost_DDR_with_noise_all = pickle.load(tf)

    h2h_ddr_ols = np.zeros(len(iteration_all)); cost_reduction_ddr_vs_ols = np.zeros(len(iteration_all)); regret_reduction_ddr_vs_ols = np.zeros(len(iteration_all))
    for iter_index in range(len(iteration_all)):
        iter = iteration_all[iter_index]
        h2h_ddr_ols[iter_index],cost_reduction_ddr_vs_ols[iter_index],regret_reduction_ddr_vs_ols[iter_index] = cross_compare2plus(cost_DDR_with_noise_all[iter,mu,lamb], cost_OLS_with_noise_all[iter], cost_Oracle_with_noise_all[iter])
    regret_p[_index] = np.round( len(regret_reduction_ddr_vs_ols[regret_reduction_ddr_vs_ols > 0.0])/len(regret_reduction_ddr_vs_ols),4 )
    h2h_p[_index] = np.round( len(h2h_ddr_ols[h2h_ddr_ols >= 0.5])/len(h2h_ddr_ols),4 )
    _index = _index + 1

/Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data/Shortest_Path_Reproduce/Various_Settings/2by2_grid_SPO_Data_Generation/data_size=100_deg=1.0_e=0.5_d=4_p=1_x_dist=uniform_num_test=1000_diff_W/
/Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data/Shortest_Path_Reproduce/Various_Settings/2by2_grid_SPO_Data_Generation/data_size=100_deg=1.0_e=0.5_d=4_p=3_x_dist=uniform_num_test=1000_diff_W/
/Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data/Shortest_Path_Reproduce/Various_Settings/2by2_grid_SPO_Data_Generation/data_size=100_deg=1.0_e=0.5_d=4_p=5_x_dist=uniform_num_test=1000_diff_W/
/Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data/Shortest_Path_Reproduce/Various_Settings/2by2_grid_SPO_Data_Generation/data_size=100_deg=1.0_e=0.5_d=4_p=7_x_dist=uniform_num_test=1000_diff_W/
/Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data/Shortest_Path_Reproduce/Various_Settings/2by2_grid_SPO_Data_Generation/data_siz

/var/folders/m2/j39mx9854cbb70jqvn7cjcqh0000gn/T/ipykernel_31586/3823106965.py:19: RuntimeWarning: invalid value encountered in scalar divide
  regret_reduction = (np.mean(c_diff))/np.abs(np.mean(c_base) - np.mean(c_oracle))
/var/folders/m2/j39mx9854cbb70jqvn7cjcqh0000gn/T/ipykernel_31586/3823106965.py:19: RuntimeWarning: divide by zero encountered in scalar divide
  regret_reduction = (np.mean(c_diff))/np.abs(np.mean(c_base) - np.mean(c_oracle))


### Impact of alpha

In [13]:
deg = 1.0 # polynomial degree
mis = deg # model misspecification

num_train = 100

num_feat = 5
p = num_feat

h2h_ddr_vs_ols_all = {}; cost_reduction_ddr_vs_ols_all = {}; regret_reduction_ddr_vs_ols_all = {}
e_all = [0.25,0.5,0.75,1.0]
for e in e_all:
    alpha = e # scale of normal std or the range of uniform. For the error term
    if e != 0.5:
        DataPath = DataPath_parent + "data_size="+str(num_train)+"_deg="+str(deg)+"_e="+str(e)+"_d="+str(d)+"_p="+str(p)+"_x_dist="+x_dist+"_num_test="+str(num_test)+"_diff_W/"
        pathlib.Path(DataPath).mkdir(parents=True, exist_ok=True)
        print("DataPath:", DataPath)
        x_test_all, c_test_all, x_train_all, c_train_all,noise_train_all,noise_test_all,W_star_all = Prepare_Data(DataPath,lower, upper, p, d, coef_seed,iteration_all,num_test, num_train, alpha,mis,data_generation_process,x_dist, e_dist, x_low, x_up, x_mean, x_var, bump)
        cost_Oracle_with_noise_all,cost_Oracle_wo_noise_all = Implement_Oracle(arcs, grid,mis,bump,W_star_all,x_test_all,noise_test_all,iteration_all,num_feat,data_generation_process)
        cost_OLS_with_noise_all,cost_OLS_wo_noise_all = Implement_OLS(arcs, grid,mis,bump,W_star_all,x_test_all,noise_test_all,x_train_all,c_train_all,iteration_all,num_feat,data_generation_process)
        cost_DDR_with_noise_all,cost_DDR_wo_noise_all = Implement_DDR(mu_all,lamb_all,arcs, grid,mis,bump,W_star_all,x_test_all,noise_test_all,x_train_all,c_train_all,iteration_all,num_feat,data_generation_process)
        with open(DataPath+'cost_OLS_with_noise_all.pkl', "wb") as tf:
            pickle.dump(cost_OLS_with_noise_all,tf)
        with open(DataPath+'cost_Oracle_with_noise_all.pkl', "wb") as tf:
            pickle.dump(cost_Oracle_with_noise_all,tf)
        with open(DataPath+'cost_DDR_with_noise_all.pkl', "wb") as tf:
            pickle.dump(cost_DDR_with_noise_all,tf)

DataPath: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data/Shortest_Path_Reproduce/Various_Settings/2by2_grid_SPO_Data_Generation/data_size=100_deg=1.0_e=0.25_d=4_p=5_x_dist=uniform_num_test=1000_diff_W/
Oracle: iter= 20 ,cost_Oracle_with_noise_all= 7.64734154101815
Oracle: iter= 40 ,cost_Oracle_with_noise_all= 7.748633761618155
Oracle: iter= 60 ,cost_Oracle_with_noise_all= 7.750990256031185
Oracle: iter= 80 ,cost_Oracle_with_noise_all= 7.590470042051599
OLS: iter= 20 ,cost_OLS_with_noise_all = 7.64915485516157
OLS: iter= 40 ,cost_OLS_with_noise_all = 7.76667460762122
OLS: iter= 60 ,cost_OLS_with_noise_all = 7.776003403176407
OLS: iter= 80 ,cost_OLS_with_noise_all = 7.596040847544069
DDR: iter= 20 ,mu= 0.5 ,lamb= 0.3 ,cost_DDR_with_noise_all = 7.649324676787289
DDR: iter= 40 ,mu= 0.5 ,lamb= 0.3 ,cost_DDR_with_noise_all = 7.7657261930022665
DDR: iter= 60 ,mu= 0.5 ,lamb= 0.3 ,cost_DDR_with_noise_all = 7.776053805825824
DDR: iter= 80 ,mu= 0.5 ,lamb= 0.3 ,cost_DDR_with_

In [14]:
mu = mu_all[0]
lamb = lamb_all[0]

regret_e = np.zeros(len(num_feat_all))
h2h_e = np.zeros(len(num_feat_all))
_index = 0
for e in e_all:
    alpha = e
    DataPath = DataPath_parent + "data_size="+str(num_train)+"_deg="+str(deg)+"_e="+str(e)+"_d="+str(d)+"_p="+str(p)+"_x_dist="+x_dist+"_num_test="+str(num_test)+"_diff_W/"
    print(DataPath)
    with open(DataPath+'cost_OLS_with_noise_all.pkl', "rb") as tf:
        cost_OLS_with_noise_all = pickle.load(tf)
    with open(DataPath+'cost_Oracle_with_noise_all.pkl', "rb") as tf:
        cost_Oracle_with_noise_all = pickle.load(tf)
    with open(DataPath+'cost_DDR_with_noise_all.pkl', "rb") as tf:
        cost_DDR_with_noise_all = pickle.load(tf)

    h2h_ddr_ols = np.zeros(len(iteration_all)); cost_reduction_ddr_vs_ols = np.zeros(len(iteration_all)); regret_reduction_ddr_vs_ols = np.zeros(len(iteration_all))
    for iter_index in range(len(iteration_all)):
        iter = iteration_all[iter_index]
        h2h_ddr_ols[iter_index],cost_reduction_ddr_vs_ols[iter_index],regret_reduction_ddr_vs_ols[iter_index] = cross_compare2plus(cost_DDR_with_noise_all[iter,mu,lamb], cost_OLS_with_noise_all[iter], cost_Oracle_with_noise_all[iter])
    regret_e[_index] = np.round( len(regret_reduction_ddr_vs_ols[regret_reduction_ddr_vs_ols > 0.0])/len(regret_reduction_ddr_vs_ols),4 )
    h2h_e[_index] = np.round( len(h2h_ddr_ols[h2h_ddr_ols >= 0.5])/len(h2h_ddr_ols),4 )
    _index = _index + 1

/Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data/Shortest_Path_Reproduce/Various_Settings/2by2_grid_SPO_Data_Generation/data_size=100_deg=1.0_e=0.25_d=4_p=5_x_dist=uniform_num_test=1000_diff_W/
/Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data/Shortest_Path_Reproduce/Various_Settings/2by2_grid_SPO_Data_Generation/data_size=100_deg=1.0_e=0.5_d=4_p=5_x_dist=uniform_num_test=1000_diff_W/
/Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data/Shortest_Path_Reproduce/Various_Settings/2by2_grid_SPO_Data_Generation/data_size=100_deg=1.0_e=0.75_d=4_p=5_x_dist=uniform_num_test=1000_diff_W/
/Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data/Shortest_Path_Reproduce/Various_Settings/2by2_grid_SPO_Data_Generation/data_size=100_deg=1.0_e=1.0_d=4_p=5_x_dist=uniform_num_test=1000_diff_W/


### Impact of model misspecification

In [15]:
e = 0.5
alpha = e

num_train = 100

num_feat = 5
p = num_feat

h2h_ddr_vs_ols_all = {}; cost_reduction_ddr_vs_ols_all = {}; regret_reduction_ddr_vs_ols_all = {}
deg_all = [0.4,0.6,0.8,1.0,1.2,1.4,1.6,2.0,3.0]
for deg in deg_all:
    mis = deg # model misspecification

    if deg != 1.0:
        DataPath = DataPath_parent + "data_size="+str(num_train)+"_deg="+str(deg)+"_e="+str(e)+"_d="+str(d)+"_p="+str(p)+"_x_dist="+x_dist+"_num_test="+str(num_test)+"_diff_W/"
        pathlib.Path(DataPath).mkdir(parents=True, exist_ok=True)
        print("DataPath:", DataPath)
        x_test_all, c_test_all, x_train_all, c_train_all,noise_train_all,noise_test_all,W_star_all = Prepare_Data(DataPath,lower, upper, p, d, coef_seed,iteration_all,num_test, num_train, alpha,mis,data_generation_process,x_dist, e_dist, x_low, x_up, x_mean, x_var, bump)
        cost_Oracle_with_noise_all,cost_Oracle_wo_noise_all = Implement_Oracle(arcs, grid,mis,bump,W_star_all,x_test_all,noise_test_all,iteration_all,num_feat,data_generation_process)
        cost_OLS_with_noise_all,cost_OLS_wo_noise_all = Implement_OLS(arcs, grid,mis,bump,W_star_all,x_test_all,noise_test_all,x_train_all,c_train_all,iteration_all,num_feat,data_generation_process)
        cost_DDR_with_noise_all,cost_DDR_wo_noise_all = Implement_DDR(mu_all,lamb_all,arcs, grid,mis,bump,W_star_all,x_test_all,noise_test_all,x_train_all,c_train_all,iteration_all,num_feat,data_generation_process)
        with open(DataPath+'cost_OLS_with_noise_all.pkl', "wb") as tf:
            pickle.dump(cost_OLS_with_noise_all,tf)
        with open(DataPath+'cost_Oracle_with_noise_all.pkl', "wb") as tf:
            pickle.dump(cost_Oracle_with_noise_all,tf)
        with open(DataPath+'cost_DDR_with_noise_all.pkl', "wb") as tf:
            pickle.dump(cost_DDR_with_noise_all,tf)

DataPath: /Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data/Shortest_Path_Reproduce/Various_Settings/2by2_grid_SPO_Data_Generation/data_size=100_deg=0.4_e=0.5_d=4_p=5_x_dist=uniform_num_test=1000_diff_W/
Oracle: iter= 20 ,cost_Oracle_with_noise_all= 5.012919335389684
Oracle: iter= 40 ,cost_Oracle_with_noise_all= 5.065063207018836
Oracle: iter= 60 ,cost_Oracle_with_noise_all= 5.029548059945788
Oracle: iter= 80 ,cost_Oracle_with_noise_all= 5.004085823269181
OLS: iter= 20 ,cost_OLS_with_noise_all = 5.047611501245823
OLS: iter= 40 ,cost_OLS_with_noise_all = 5.116822698930861
OLS: iter= 60 ,cost_OLS_with_noise_all = 5.069416452196432
OLS: iter= 80 ,cost_OLS_with_noise_all = 5.0287257720215175
DDR: iter= 20 ,mu= 0.5 ,lamb= 0.3 ,cost_DDR_with_noise_all = 5.05212144731228
DDR: iter= 40 ,mu= 0.5 ,lamb= 0.3 ,cost_DDR_with_noise_all = 5.1137289243554
DDR: iter= 60 ,mu= 0.5 ,lamb= 0.3 ,cost_DDR_with_noise_all = 5.071090274253859
DDR: iter= 80 ,mu= 0.5 ,lamb= 0.3 ,cost_DDR_with_n

In [16]:
mu = mu_all[0]
lamb = lamb_all[0]

regret_deg = np.zeros(len(num_feat_all))
h2h_deg = np.zeros(len(num_feat_all))
_index = 0
for deg in deg_all:
    mis = deg # model misspecification
    DataPath = DataPath_parent + "data_size="+str(num_train)+"_deg="+str(deg)+"_e="+str(e)+"_d="+str(d)+"_p="+str(p)+"_x_dist="+x_dist+"_num_test="+str(num_test)+"_diff_W/"
    print(DataPath)
    with open(DataPath+'cost_OLS_with_noise_all.pkl', "rb") as tf:
        cost_OLS_with_noise_all = pickle.load(tf)
    with open(DataPath+'cost_Oracle_with_noise_all.pkl', "rb") as tf:
        cost_Oracle_with_noise_all = pickle.load(tf)
    with open(DataPath+'cost_DDR_with_noise_all.pkl', "rb") as tf:
        cost_DDR_with_noise_all = pickle.load(tf)

    h2h_ddr_ols = np.zeros(len(iteration_all)); cost_reduction_ddr_vs_ols = np.zeros(len(iteration_all)); regret_reduction_ddr_vs_ols = np.zeros(len(iteration_all))
    for iter_index in range(len(iteration_all)):
        iter = iteration_all[iter_index]
        h2h_ddr_ols[iter_index],cost_reduction_ddr_vs_ols[iter_index],regret_reduction_ddr_vs_ols[iter_index] = cross_compare2plus(cost_DDR_with_noise_all[iter,mu,lamb], cost_OLS_with_noise_all[iter], cost_Oracle_with_noise_all[iter])
    regret_deg[_index] = np.round( len(regret_reduction_ddr_vs_ols[regret_reduction_ddr_vs_ols > 0.0])/len(regret_reduction_ddr_vs_ols),4 )
    h2h_deg[_index] = np.round( len(h2h_ddr_ols[h2h_ddr_ols >= 0.5])/len(h2h_ddr_ols),4 )
    _index = _index + 1

/Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data/Shortest_Path_Reproduce/Various_Settings/2by2_grid_SPO_Data_Generation/data_size=100_deg=0.4_e=0.5_d=4_p=5_x_dist=uniform_num_test=1000_diff_W/
/Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data/Shortest_Path_Reproduce/Various_Settings/2by2_grid_SPO_Data_Generation/data_size=100_deg=0.6_e=0.5_d=4_p=5_x_dist=uniform_num_test=1000_diff_W/
/Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data/Shortest_Path_Reproduce/Various_Settings/2by2_grid_SPO_Data_Generation/data_size=100_deg=0.8_e=0.5_d=4_p=5_x_dist=uniform_num_test=1000_diff_W/
/Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data/Shortest_Path_Reproduce/Various_Settings/2by2_grid_SPO_Data_Generation/data_size=100_deg=1.0_e=0.5_d=4_p=5_x_dist=uniform_num_test=1000_diff_W/
/Users/zhangxun/Dropbox/Research/Decision_Driven_Regularization/Data/Shortest_Path_Reproduce/Various_Settings/2by2_grid_SPO_Data_Generation/data_siz

IndexError: index 6 is out of bounds for axis 0 with size 6

In [ ]:
# import matplotlib.pyplot as plt
# fig = plt.figure(figsize = (5, 5))
# # plt.plot(h2h_p, regret_p, color='green', marker = "*", label = 'p', linestyle = 'None')
# # plt.text(h2h_p[0] + 0.01, regret_p[0] - 0.005, 'Baseline', color='red')
# # plt.text(h2h_p[1] + 0.01, regret_p[1] - 0.005, 'p = 6', color='green')
# # plt.text(h2h_p[2] - 0.05, regret_p[2] - 0.005, 'p = 8', color='green')
# # plt.text(h2h_p[3] - 0.065, regret_p[3] - 0.005, 'p = 10', color='green')

# # plt.plot(h2h_d[0], regret_d[0], color='#003D7C', marker = "o", label = 'd', linestyle = 'None')
# # # plt.plot(h2h_d[1], mci_d[1], color='#003D7C', marker = "o", linestyle = 'None')
# # plt.plot(h2h_d[2], regret_d[2], color='#003D7C', marker = "o", linestyle = 'None')
# # plt.plot(h2h_d[3], regret_d[3], color='#003D7C', marker = "o", linestyle = 'None')
# # plt.text(h2h_d[0] + 0.01, regret_d[0] - 0.005, 'd = 5', color='#003D7C')
# # # plt.text(h2h_d[1] + 0.007, mci_d[1] - 0.003, 'd = 15', color='#003D7C')
# # plt.text(h2h_d[2] + 0.01, regret_d[2] - 0.005, 'd = 25', color='#003D7C')
# # plt.text(h2h_d[3] - 0.065, regret_d[3] - 0.005, 'd = 50', color='#003D7C')


# plt.plot(h2h_N[0], regret_N[0], color='#EF7C00', marker = "^", label = 'N',linestyle = 'None')
# plt.text(h2h_N[1] + 0.01, regret_N[1] - 0.005, 'Baseline', color='red')
# # plt.plot(h2h_N[1], mci_N[1], color='#EF7C00', marker = "^",linestyle = 'None')
# plt.plot(h2h_N[2], regret_N[2], color='#EF7C00', marker = "^",linestyle = 'None')
# plt.plot(h2h_N[3], regret_N[3], color='#EF7C00', marker = "^",linestyle = 'None')
# plt.plot(h2h_N[4], regret_N[4], color='#EF7C00', marker = "^",linestyle = 'None')
# plt.text(h2h_N[0] - 0.02, regret_N[0] + 0.005, 'N = 50', color='#EF7C00')
# # plt.text(h2h_N[1] + 0.005, mci_N[1] - 0.003, 'N = 100', color='#EF7C00')
# plt.text(h2h_N[2] - 0.075, regret_N[2] - 0.005, 'N = 200', color='#EF7C00')
# plt.text(h2h_N[3] - 0.075, regret_N[3] - 0.005, 'N = 500', color='#EF7C00')
# plt.text(h2h_N[4] - 0.085, regret_N[4] - 0.005, 'N = 1000', color='#EF7C00')


# # plt.plot(h2h_alpha[0], regret_alpha[0], color='grey', marker = "d", label = r'$\alpha$',linestyle = 'None')
# # # plt.plot(h2h_alpha[1], mci_alpha[1], color='grey', marker = "d",linestyle = 'None')
# # plt.plot(h2h_alpha[2], regret_alpha[2], color='grey', marker = "d",linestyle = 'None')
# # plt.plot(h2h_alpha[3], regret_alpha[3], color='grey', marker = "d",linestyle = 'None')
# # plt.text(h2h_alpha[0] + 0.01, regret_alpha[0] - 0.005, r'$\alpha$ = 0.25', color='grey')
# # # plt.text(h2h_alpha[1] + 0.005, mci_alpha[1] - 0.005, r'$\alpha$ = 1', color='grey')
# # plt.text(h2h_alpha[2] - 0.075, regret_alpha[2] - 0.005, r'$\alpha$ = 2.25', color='grey')
# # plt.text(h2h_alpha[3] - 0.05, regret_alpha[3] - 0.005, r'$\alpha$ = 4', color='grey')

# plt.vlines(0.5, 0.0, 0.8, linestyle="dashed", alpha = 0.8,color = 'k')
# plt.hlines(0.0, 0.4, 0.8, linestyle="dashed", alpha = 0.8,color = 'k')
# plt.legend()

# plt.xlabel('Head-to-head')
# plt.ylabel('Regret reduction')

# plt.plot(h2h_N[1], regret_N[1], color='red', marker = "o", linestyle = 'None')
# # plt.annotate('Ridge vs. DDR', xy = (0.25,0.9), xycoords = 'axes fraction', bbox=dict(boxstyle="round", fc="w"), size = 10)

# # fig.savefig(DataPath+'DDR_vs_OLS_DPSN.eps', format='eps', bbox_inches="tight")
# # fig.savefig(DataPath+'DDR_vs_OLS_DPSN.pdf', format='pdf', bbox_inches="tight")
# plt.show()